<a href="https://www.kaggle.com/code/lyhaiyen/predictpricehousing?scriptVersionId=112668964" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/Melbourne_housing_FULL_dataset"

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats, special
import matplotlib.pyplot as plt
import datetime as dt
from numpy import mean, std

In [ ]:
dataDF = pd.read_csv("/content/drive/MyDrive/Melbourne_housing_FULL_dataset/Melbourne_housing_FULL.csv")
print('Số lượng mẫu = %d' %(dataDF.shape[0]))
print('Số lượng thuộc tính = %d' %(dataDF.shape[1]))

In [ ]:
dataDF.info()

In [ ]:
dataDF.describe()

In [ ]:
categorical = ['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea', 'Regionname']
categorical2 = ['Rooms', 'Bedroom2', 'Bathroom', 'Car']
continous = ['Price','Distance','Lattitude','Longtitude','Propertycount','Postcode', 'Landsize', 'BuildingArea', 'YearBuilt']
columns = ['Rooms', 'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']

In [ ]:
for column_name in columns:
  plt.figure(figsize=(10,10))
  sns.histplot(dataDF[column_name])
plt.show()

In [ ]:
for col in continous:
    plt.figure(figsize=(15,10))
    sns.regplot(x = col,y = 'Price', data=dataDF)

In [ ]:
for column_name in categorical:
    plt.figure(figsize=(25,10))
    sns.boxplot(x=column_name, y='Price', data=dataDF)

In [ ]:
for column_name in categorical2:
    plt.figure(figsize=(25,10))
    sns.boxplot(x=column_name, y='Price', data=dataDF)

In [ ]:
corr = dataDF.corr()
plt.figure(figsize=(15,15))
sns.heatmap(corr, annot=True, fmt= '.2f', annot_kws={'size': 15}, cmap= 'coolwarm')
plt.show()

In [ ]:
dataDF["Suburb"] = dataDF["Suburb"].astype('category')
dataDF["Type"] = dataDF["Type"].astype('category')
dataDF["Method"] = dataDF["Method"].astype('category')
dataDF["SellerG"] = dataDF["SellerG"].astype('category')
dataDF["CouncilArea"] = dataDF["CouncilArea"].astype('category')
dataDF["Regionname"] = dataDF["Regionname"].astype('category')

In [ ]:
dataDF[categorical].describe()

In [ ]:
def missingvalues(data): 
    for col in data.columns:
        print('\t%s: %d' % (col,data[col].isna().sum()))
def MissingHandler(data):
    DataMissing = data.isnull().sum()*100/len(data)
    DataMissingByColumn = pd.DataFrame({'Tỷ lệ phần trăm dữ liệu null':DataMissing})
    DataMissingByColumn.sort_values(by='Tỷ lệ phần trăm dữ liệu null',ascending=False,inplace=True)
    return DataMissingByColumn[DataMissingByColumn['Tỷ lệ phần trăm dữ liệu null']>0]

Loại bỏ các mẫu không có giá trị price 

In [ ]:
# check NaN trong col Price và drop row có NaN Priced
dataDF = dataDF[dataDF['Price'].notna()]
dataDF = dataDF[dataDF['Distance'].notna()]
dataDF.reset_index(inplace=True)
MissingHandler(dataDF)
dataDF = dataDF.drop(['index'], axis =1)

In [ ]:
missingvalues(dataDF)

### Xử Lí Dữ Liệu Ngày Tháng

In [ ]:
# convert to DateTime
dataDF['Date'] = pd.to_datetime(dataDF['Date'])
assert dataDF['Date'].dtype == 'datetime64[ns]'

In [ ]:
dataDF['Year'] = pd.DatetimeIndex(dataDF['Date']).year
dataDF['Month'] = pd.DatetimeIndex(dataDF['Date']).month
dataDF['Day'] = pd.DatetimeIndex(dataDF['Date']).day

### Xử lý Address

In [ ]:
address = dataDF['Address'].copy()
addr = address.values
StreetAndalley  = dataDF['Address'].str.split(' ', n = 1, expand = True)[0]
index = StreetAndalley.index
for x in index:
  if(len(StreetAndalley[x].split('/'))==1):
    StreetAndalley[x] = 0
  else:
    StreetAndalley[x] = 1
StreetAndalley

In [ ]:
StreetAndalley = pd.DataFrame(StreetAndalley).astype('int')
dataDF['StreetAndalley'] = StreetAndalley

### Xử lý Suburb

In [ ]:
SuburbAndDistance = dataDF[['Suburb', 'Distance']].copy()
SuburbAndDistance['Distance'].value_counts().index.sort_values(ascending = False).values

In [ ]:
SuburbAndDistance['Suburb'] = np.where(SuburbAndDistance['Distance'] < 10, 1, SuburbAndDistance['Suburb'])
SuburbAndDistance['Suburb'] = np.where(SuburbAndDistance['Distance'] > 30, 3,SuburbAndDistance['Suburb'])
SuburbAndDistance['Suburb'] = np.where((SuburbAndDistance['Distance'].values<=30)&(SuburbAndDistance['Distance'].values >= 10), 2,SuburbAndDistance['Suburb'])
SuburbAndDistance

In [ ]:
dataDF['Suburb'] = SuburbAndDistance['Suburb']
dataDF['Suburb'] = dataDF['Suburb'].astype('int')

## XỬ LÍ MISSING DATA

In [ ]:
def replaceValueM(data):
    data['Lattitude'].fillna(data['Lattitude'].mean(), inplace = True)
    data['Longtitude'].fillna(data['Longtitude'].mean(), inplace = True)
    return MissingHandler(dataDF)
replaceValueM(dataDF)

In [ ]:
print("Các giá trị duy nhất trong Rooms : ",np.unique(dataDF['Rooms']))
print("Các giá trị duy nhất trong Bedroom2 : ",np.unique(dataDF[dataDF['Bedroom2'].notna()] ['Bedroom2']))
print("Các giá trị duy nhất trong Bathroom : ",np.unique(dataDF[dataDF['Bathroom'].notna()]['Bathroom']))
print("Các giá trị duy nhất trong Car : ",np.unique(dataDF[dataDF['Car'].notna()]['Car']))

In [ ]:
categorical3 = ['Regionname', 'CouncilArea']
for column_name in categorical3:
  a=dataDF[column_name].value_counts().head(1).index.values[0]
  dataDF[column_name]=dataDF[column_name].fillna(a)
  print(column_name,a)

In [ ]:
cl1 = ['Rooms', 'Bedroom2']
df1 = dataDF[cl1].copy()
df11 = df1.loc[df1['Bedroom2'].isna()]
df1 = df1.dropna(axis = 0)

In [ ]:
X = df1['Rooms'].values
Y = df1['Bedroom2'].values
Xtest = df11['Rooms'].values
plt.figure(figsize=(10,7))
plt.scatter(X,Y)

In [ ]:
c = df1.loc[df1['Bedroom2'].values > 19].index.values
df1 = df1.drop(index = c,axis = 0)
X = df1['Rooms'].values
Y = df1['Bedroom2'].values
plt.figure(figsize=(10,7))
plt.scatter(X,Y)

In [ ]:
from sklearn import linear_model
reg =  linear_model.LinearRegression()
reg.fit(X.reshape(-1,1),Y)

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(X,Y)
plt.plot(Xtest, reg.predict(Xtest.reshape(-1,1)))

In [ ]:
ypred = reg.predict(Xtest.reshape(-1,1))
ypred

In [ ]:
df11['Bedroom2'] = np.around(ypred)
df11

In [ ]:
dataDF['Bedroom2'] = dataDF['Bedroom2'].fillna(df11['Bedroom2'])

In [ ]:
dataDF['Bedroom2'].isna().sum()

In [ ]:
cl2 = ['Rooms', 'Bedroom2', 'Bathroom']
df2 = dataDF[cl2].copy()
df2

In [ ]:
df22 = df2.loc[df2['Bathroom'].isna()]
df2 = df2.dropna(axis = 0)
df2

In [ ]:
Xtest = df22.loc[:, 'Rooms':'Bedroom2']
X = df2.loc[:, 'Rooms':'Bedroom2']
Y = df2.loc[:, 'Bathroom']

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X,Y)

In [ ]:
ypred = regr.predict(Xtest)
ypred

In [ ]:
df22['Bathroom'] = np.around(ypred)
df22

In [ ]:
dataDF['Bathroom'] = dataDF['Bathroom'].fillna(df22['Bathroom'])
dataDF['Bathroom'].isna().sum()

In [ ]:
cl3 = ['Rooms', 'Bedroom2', 'Bathroom', 'Car']
df3 = dataDF[cl3].copy()
df3

In [ ]:
df33 = df3.loc[df3['Car'].isna()]
df3 = df3.dropna(axis = 0)
df3

In [ ]:
df3.info()

In [ ]:
Xtest = df33.loc[:, 'Rooms':'Bathroom']
X = df3.loc[:, 'Rooms':'Bathroom']
Y = df3.loc[:, 'Car']

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X,Y)

In [ ]:
ypred = regr.predict(Xtest)
ypred

In [ ]:
df33['Car'] = np.around(ypred)
df33

In [ ]:
dataDF['Car'] = dataDF['Car'].fillna(df33['Car'])

In [ ]:
MissingHandler(dataDF)

In [ ]:
dataDF['AgeOfYearBuilt'] = 2021-dataDF['YearBuilt']
dataDF['AgeOfYearBuilt'].fillna(0, inplace=True)
# dataDF['TotalArea'] = dataDF['BuildingArea']+dataDF['Landsize']
# dataDF['TotalArea'].fillna(0, inplace=True)

In [ ]:
dataDF = dataDF.drop(['BuildingArea', 'Landsize', 'YearBuilt'], axis = 1)
MissingHandler(dataDF)

In [ ]:
dataDF = dataDF.dropna()

In [ ]:
dataDF.describe()

In [ ]:
datalast = dataDF.drop([ "Address", "SellerG", "Date"], axis=1)
datalast.info()

In [ ]:
def SkewMeasure(df):
    nonObjectColList = df.dtypes[(df.dtypes != 'object') & (df.dtypes != 'category')].index
    skewM = df[nonObjectColList].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending = False)
    skewM=pd.DataFrame({'skew':skewM})
    return skewM[abs(skewM)>0.5].dropna()
skewM = SkewMeasure(datalast)
skewM

In [ ]:
for i in skewM.index:
    datalast[i] = special.boxcox1p(datalast[i], 0.15) #lambda  
SkewMeasure(datalast)

In [ ]:
datalast.hist(bins = 50, figsize=(20,20))
plt.show()

### XỬ LÍ DỮ LIỆU CATEGORY

In [ ]:
# def top_x(data,variable,top_x_labels):
#   for label in top_x_labels:
#     data[variable+'_'+label] = np.where(data[variable]==label,1,0)

In [ ]:
categorical3 = ['Type', 'Method', 'CouncilArea', 'Regionname']
for column_name in categorical3:  
  datalast = pd.get_dummies(datalast, columns=[column_name], drop_first=False)

In [ ]:
datalast

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
MinMax = MinMaxScaler()
SS = StandardScaler()
featureGaussD = ['Lattitude', 'Longtitude', 'Rooms', 'Bedroom2']
featureNonGD = ['Suburb', 'Price', 'Distance', 'Postcode', 'Bathroom', 'Car', 'Propertycount', 'Year', 'Month', 'Day', 'StreetAndalley', 'AgeOfYearBuilt']
datalast[featureGaussD] = SS.fit_transform(datalast[featureGaussD])
datalast[featureNonGD] = MinMax.fit_transform(datalast[featureNonGD])

In [ ]:
Price = datalast['Price']
datalast = datalast.drop(['Price'], axis=1)

In [ ]:
from sklearn import metrics
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
xTrain, xTest, yTrain, yTest = model_selection.train_test_split(datalast, Price,test_size=0.2,random_state=42)
linearmodel = LinearRegression()
linearmodel.fit(xTrain, yTrain)

In [ ]:
y_predict = linearmodel.predict(xTest)
y_predict

In [ ]:
yTest

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(yTest,y_predict)
plt.title('So sánh giá trị thật và giá trị dự đoán của tập test')
plt.xlabel('Giá trị thật của Price')
plt.ylabel('Giá trị dự đoán của Price')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.plot([0, 1], [0, 1])

In [ ]:
plt.figure(figsize=(20,20))
ax = sns.kdeplot(yTest)
ax = sns.kdeplot(y_predict, ax=ax)
ax.legend(['y', 'y_predict'], fontsize=13);

In [ ]:
print("Root mean squared error = %.4f" % np.sqrt(mean_squared_error(yTest, y_predict)))

In [ ]:
 print(f"R^2:\t{r2_score(yTest, y_predict):.3f}")
 print('R-squared = %.4f' % r2_score(yTest, y_predict))

In [ ]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
X_train, X_test, y_train, y_test = model_selection.train_test_split(datalast, Price, test_size=0.2, random_state=42)
model = keras.Sequential()
model.add(layers.Dense(1, input_shape = (68,)))
model.add(layers.Activation('linear'))
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
model.compile(loss='mse', optimizer = optimizer, metrics=['mse', 'mae', coeff_determination])
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2)
history_df = pd.DataFrame(history.history)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(history_df['loss'], label='loss')
plt.plot(history_df['val_loss'], label='val_loss')
plt.legend()

In [ ]:
ypred = model.predict(X_test)

In [ ]:
m = keras.metrics.RootMeanSquaredError()
m.update_state(y_test, ypred)
result = m.result()
result.numpy()

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(y_test,ypred)
plt.title('So sánh giá trị thật và giá trị dự đoán của tập test')
plt.xlabel('Giá trị thật của Price')
plt.ylabel('Giá trị dự đoán của Price')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.plot([0, 1], [0, 1])

In [ ]:
y_pred = pd.DataFrame(ypred, columns=['predicted_price'])
plt.figure(figsize=(15,15))
ax = sns.kdeplot(y_test)
ax = sns.kdeplot(y_pred['predicted_price'], ax=ax)
ax.legend(['y', 'y_predict'], fontsize=13);